In [1]:
using ITensors

In [ ]:
function compress_expansion(C::NormalizedTensor, P::NormalizedTensor, chi::Int64)
    z, w = inds(C.tensor)

    D, U = eigen(C.tensor, z, w)

    n = dim(z)

    p = Index(chi, "p")
    q = Index(chi, "q")

    C_compressed = ITensor(p, q)

    for ii in 1:chi, jj in 1:chi
        C_compressed[p=>ii, q=>jj] = D[inds(D)[1]=>n-ii+1, inds(D)[2]=>n-jj+1]
    end

    NC_compressed = normalize_tensor(C_compressed)
    norm_C = C.norm * NC_compressed.norm

    inds_U = inds(U)
    NU = normalize_tensor(U)

    u = Index(dim(inds(U)[1]), "u")
    v = Index(chi, "v")

    U_compressed = ITensor(u, v)

    for ii in 1:n, jj in 1:chi
        U_compressed[u=>ii, v=>jj] = NU.tensor[inds_U[1]=>n-ii+1, inds_U[2]=>n-jj+1]
    end

    inds_U_compressed = inds(U_compressed)
    inds_P = inds(P.tensor)
    n_u = dim(inds_U_compressed[1])
    m_u = dim(inds_U_compressed[2])

    a = Index(n_u, "a")
    b = Index(m_u, "b")
    c = Index(n_u, "c")
    d = Index(m_u, "d")
    e = Index(dim(inds_P[1]), "e")

    inds_new_U_compressed_l = [a, b]
    inds_new_U_compressed_r = [c, d]
    inds_new_P = [e, a, c]

    U_compressed_l = replaceinds(U_compressed, inds_U_compressed, inds_new_U_compressed_l)
    U_compressed_r = replaceinds(U_compressed, inds_U_compressed, inds_new_U_compressed_r)
    P_b = replaceinds(P.tensor, inds_P, inds_new_P)

    P_compressed = P_b * U_compressed_l * U_compressed_r
    return NormalizedTensor(NC_compressed.tensor, norm_C), NormalizedTensor(NP_compressed.tensor, norm_P)
end